In [1]:
# Install required libraries
!pip install transformers datasets accelerate

In [3]:
import json

with open("Ecommerce_FAQ_Chatbot_dataset.json", "r") as f:
  data = json.load(f)

questions = data["questions"]
answers = [item["answer"] for item in questions]


In [4]:
def format_to_chatml(question, answer):
  return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n{answer}<|im_end|>\n"

train_data = [format_to_chatml(q, a) for q, a in zip(questions, answers)]


In [5]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
import torch

# Load model and tokenizer (assuming the model exists)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Replace with your chosen TinyLlama model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Train model function (adjust hyperparameters as needed)
def train_model(train_data, epochs=3):
  optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

  for epoch in range(epochs):
    for i, sample in enumerate(train_data):
      # Input preparation
      input_ids = tokenizer(sample, return_tensors="pt")["input_ids"]
      labels = tokenizer(sample, return_tensors="pt")["input_ids"]

      # Create input dictionary for the model
      input_dict = {"input_ids": input_ids, "labels": labels}

      # Training loop using the input dictionary
      outputs = model(**input_dict)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if i % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Step {i}: Loss = {loss.item():.4f}")

  return model

# Train the model (assuming you have prepared your train_data)
trained_model = train_model(train_data)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Epoch 1/3, Step 0: Loss = 1.4314


KeyboardInterrupt: 